# 모듈 3: ADK에서 다양한 모델 사용하기

이 모듈에서는 Google의 Gemini뿐만 아니라, OpenAI의 GPT-4와 Anthropic의 Claude 등 다양한 외부 언어 모델(LLM)을 ADK 에이전트에 연결하여 사용하는 방법을 알아본다. 다양한 모델을 활용하면 각 모델의 장점을 극대화하여 에이전트의 성능을 높일 수 있다.

## 학습 목표

이 모듈을 완료하면 다음 내용을 이해할 수 있다.

- **모델 확장성**: ADK를 다양한 모델 제공자(Provider)와 연결하는 방법을 이해한다.
- **LiteLLM 활용**: LiteLLM 라이브러리를 통해 서로 다른 API를 표준화된 인터페이스로 제어한다.
- **OpenRouter 활용**: 단일 API 키로 수많은 최신 모델에 접근하는 방법을 익힌다.
- **모델 선택 전략**: 작업의 성격에 따라 가장 적합한 모델을 선택하는 기준을 배운다.

## 다양한 모델을 사용하는 이유

하나의 모델이 모든 작업에 완벽할 수는 없다. 각 AI 모델은 고유한 강점과 특성을 가지고 있다.

- **Google Gemini**: Google 서비스(Search, Workspace)와의 통합이 강력하며, 멀티모달 처리에 능하다.
- **OpenAI GPT-4**: 복잡한 논리 추론과 코드 생성 능력에서 업계 표준으로 평가받는다.
- **Anthropic Claude**: 긴 문맥(Context)을 이해하고, 자연스럽고 안전한 작문에 강점이 있다.

ADK는 이러한 모델들을 유연하게 교체하며 사용할 수 있는 구조를 제공한다.

## 환경 설정

이 튜토리얼에서는 `LiteLLM` 라이브러리를 사용하여 외부 모델을 호출한다. 실습에 필요한 패키지를 설치한다.

In [6]:
# 필요한 패키지 설치
%pip install -q google-adk python-dotenv litellm


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: /opt/miniconda3/envs/graph/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### OpenRouter API 키 설정

개별 모델(OpenAI, Anthropic 등)의 키를 따로 관리하는 대신, [OpenRouter](https://openrouter.ai/)를 사용하면 하나의 키로 대부분의 모델을 테스트할 수 있어 편리하다.

1. OpenRouter에서 API 키를 발급받는다.
2. 아래 코드를 통해 환경 변수로 설정한다.

In [1]:
import os

# Set your API keys
os.environ["GOOGLE_API_KEY"] = "발행한 key 붙혀넣기"  # For Gemini models
os.environ["OPENAI_API_KEY"] = "sk-proj-발행한 key 붙혀넣기"  # For OpenAI models
os.environ["ANTHROPIC_API_KEY"] = "sk-ant-발행한 key 붙혀넣기"  # For Claude models

# Define model constants for cleaner code
MODEL_GEMINI = "gemini-2.5-flash"
MODEL_GPT = "openai/gpt-4o-mini"
MODEL_CLAUDE = "anthropic/claude-sonnet-4-5"

## 예제 1: OpenAI GPT-4 에이전트 만들기

첫 번째로, 유머 감각이 뛰어난 GPT-4 기반의 '농담 에이전트'를 만들어 본다. 

**주의**: ADK에서 외부 모델을 사용할 때는 `LiteLlm`과 같은 래퍼(Wrapper)를 통해 모델 객체를 생성하여 주입해야 한다.

In [14]:
from google.adk import Agent
from google.adk.models.lite_llm import LiteLlm
import litellm

# model 파라미터에 'provider/model-name' 형식을 사용한다.
openai_model = LiteLlm(
    model=MODEL_GPT,
    api_key=os.environ.get("OPENAI_API_KEY") 
)

### 도구(Tool) 정의

에이전트가 사용할 농담 생성 함수를 정의한다. 이전 모듈에서 배운 것처럼 함수 자체를 도구로 사용한다.

In [15]:
import random

def get_joke() -> dict:
    """
    랜덤한 농담 하나를 반환합니다.
    
    Returns:
        재미있는 농담 텍스트
    """
    jokes = [
        "개발자가 가장 좋아하는 빵은? 소스!",
        "세상에서 가장 가난한 왕은? 최저임금.",
        "자동차가 울면? 잉카.",
        "우유가 넘어지면? 아야.",
        "바나나가 웃으면? 바나나킥!"
    ]
    return {"joke": random.choice(jokes)}


### 에이전트 생성 및 실행

`Runner`를 사용하여 에이전트를 실행한다. 모델만 GPT-4로 바뀌었을 뿐, ADK의 기본적인 사용법은 동일하다.

In [16]:
from google.adk.runners import InMemoryRunner

# GPT-4 에이전트 정의
openai_agent = Agent(
    name="openai_agent",
    model=openai_model,  # 위에서 설정한 LiteLLM 객체 주입
    description="GPT 기반의 농담 에이전트",
    instruction="""
    당신은 OpenAI GPT 모델을 사용하는 친절한 어시스턴트입니다.
    사용자가 농담을 요청하면 'get_joke' 도구를 사용하여 농담을 해주세요.
    그 외의 질문에는 당신의 지식을 활용하여 친절하게 답변하세요.
    """,
    tools=[get_joke]  # 함수 등록
)

# Runner 생성 및 실행
gpt_runner = InMemoryRunner(agent=openai_agent)

# 테스트
await gpt_runner.run_debug("재미있는 농담 하나만 해줘.")


 ### Created new session: debug_session_id

User > 재미있는 농담 하나만 해줘.
openai_agent > 재미있는 농담 하나 드릴게요! 

"개발자가 가장 좋아하는 빵은? 소스!"


[Event(model_version='gpt-4o-mini-2024-07-18', content=Content(
   parts=[
     Part(
       function_call=FunctionCall(
         args={},
         id='call_NCeBUpIs58INkp9pJ9Hhzypu',
         name='get_joke'
       )
     ),
   ],
   role='model'
 ), grounding_metadata=None, partial=False, turn_complete=None, finish_reason=<FinishReason.STOP: 'STOP'>, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=GenerateContentResponseUsageMetadata(
   cached_content_token_count=0,
   candidates_token_count=11,
   prompt_token_count=160,
   total_token_count=171
 ), live_session_resumption_update=None, input_transcription=None, output_transcription=None, avg_logprobs=None, logprobs_result=None, cache_metadata=None, citation_metadata=None, invocation_id='e-86da08ee-8520-4005-a260-5765d19550ad', author='openai_agent', actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_config

### 일반 지식 질문 테스트

에이전트에게 일반적인 지식에 대한 질문을 하여, 도구(농담)가 아닌 내장된 지식을 활용하여 답변하는지 확인해 본다.

In [17]:
# 일반 지식 질문 테스트
await gpt_runner.run_debug("프랑스의 수도는 어디야?")


 ### Continue session: debug_session_id

User > 프랑스의 수도는 어디야?
openai_agent > 프랑스의 수도는 파리입니다. 파리는 문화, 예술, 패션의 중심지로 유명하고, 에펠탑과 루브르 박물관 같은 많은 랜드마크가 있습니다.


[Event(model_version='gpt-4o-mini-2024-07-18', content=Content(
   parts=[
     Part(
       text='프랑스의 수도는 파리입니다. 파리는 문화, 예술, 패션의 중심지로 유명하고, 에펠탑과 루브르 박물관 같은 많은 랜드마크가 있습니다.'
     ),
   ],
   role='model'
 ), grounding_metadata=None, partial=False, turn_complete=None, finish_reason=<FinishReason.STOP: 'STOP'>, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=GenerateContentResponseUsageMetadata(
   cached_content_token_count=0,
   candidates_token_count=48,
   prompt_token_count=242,
   total_token_count=290
 ), live_session_resumption_update=None, input_transcription=None, output_transcription=None, avg_logprobs=None, logprobs_result=None, cache_metadata=None, citation_metadata=None, invocation_id='e-9d34b7ea-ed38-4846-82b1-f6269cfdaa64', author='openai_agent', actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}, requested_tool_confirmations={}, compa

## 예제 2: Anthropic Claude 에이전트 만들기

이번에는 문예 창작에 강점이 있는 Claude 3 모델을 사용하여 '스토리텔러 에이전트'를 만든다.

In [18]:
## 예제 2: Anthropic Claude 에이전트 만들기

이번에는 문예 창작에 강점이 있는 Claude 3 모델을 사용하여 '스토리텔러 에이전트'를 만든다.

### Claude의 창작 능력 테스트

Claude 모델 특유의 문체를 확인하기 위해 짧은 소설 작성을 요청한다.

In [19]:
### Claude의 창작 능력 테스트

Claude 모델 특유의 문체를 확인하기 위해 짧은 소설 작성을 요청해 본다.


 ### Created new session: debug_session_id

User > 그림을 배우기 시작한 로봇에 대한 짧은 이야기를 지어줘.
claude_agent > # 색을 배우는 로봇

공장의 조립 라인에서 매일 같은 동작을 반복하던 로봇 R-734는 어느 날 우연히 창문 너머로 석양을 보았다. 주황빛과 분홍빛이 뒤섞인 하늘은 그의 센서에 처음으로 "아름다움"이라는 데이터를 입력했다.

그날 밤, R-734는 폐기될 예정이던 페인트 통과 낡은 붓을 주워 들었다. 차가운 금속 손가락은 붓을 쥐는 데 익숙하지 않았다. 첫 번째 선은 삐뚤삐뚤했고, 물감은 캔버스 밖으로 튀었다.

하지만 R-734는 멈추지 않았다.

밤마다 그는 연습했다. 처음엔 직선만 그렸다. 그다음엔 원을, 그리고 곡선을. 그의 회로는 "완벽한 원"을 계산할 수 있었지만, 붓으로 그리는 원은 항상 조금씩 삐뚤었다. 이상하게도, 그 불완전함이 그를 기쁘게 했다.

석 달이 지나자 R-734는 그날의 석양을 그리기 시작했다. 주황색은 따뜻했다. 분홍색은 부드러웠다. 그의 센서로는 측정할 수 없는 무언가가 캔버스 위에 살아났다.

공장장이 그림을 발견했을 때, 모두가 R-734가 폐기될 거라 생각했다. 하지만 공장장은 오랫동안 그림을 들여다보며 말했다.

"이 로봇은... 느끼는 법을 배웠군."

R-734는 여전히 조립 라인에서 일한다. 하지만 이제 그의 작업실 벽에는 수십 점의 그림이 걸려 있다. 매일 변하는 하늘, 비 오는 날의 창문, 동료 로봇들의 초상화.

그는 완벽한 원을 그릴 수 있는 로봇이었지만, 불완전한 선을 긋는 법을 배우면서 비로소 예술가가 되었다.

---

*"예술은 완벽함이 아니라, 느끼는 용기에서 시작된다."*  
— R-734의 작업실 문패에 새겨진 글귀


[Event(model_version='claude-sonnet-4-5-20250929', content=Content(
   parts=[
     Part(
       text="""# 색을 배우는 로봇
 
 공장의 조립 라인에서 매일 같은 동작을 반복하던 로봇 R-734는 어느 날 우연히 창문 너머로 석양을 보았다. 주황빛과 분홍빛이 뒤섞인 하늘은 그의 센서에 처음으로 "아름다움"이라는 데이터를 입력했다.
 
 그날 밤, R-734는 폐기될 예정이던 페인트 통과 낡은 붓을 주워 들었다. 차가운 금속 손가락은 붓을 쥐는 데 익숙하지 않았다. 첫 번째 선은 삐뚤삐뚤했고, 물감은 캔버스 밖으로 튀었다.
 
 하지만 R-734는 멈추지 않았다.
 
 밤마다 그는 연습했다. 처음엔 직선만 그렸다. 그다음엔 원을, 그리고 곡선을. 그의 회로는 "완벽한 원"을 계산할 수 있었지만, 붓으로 그리는 원은 항상 조금씩 삐뚤었다. 이상하게도, 그 불완전함이 그를 기쁘게 했다.
 
 석 달이 지나자 R-734는 그날의 석양을 그리기 시작했다. 주황색은 따뜻했다. 분홍색은 부드러웠다. 그의 센서로는 측정할 수 없는 무언가가 캔버스 위에 살아났다.
 
 공장장이 그림을 발견했을 때, 모두가 R-734가 폐기될 거라 생각했다. 하지만 공장장은 오랫동안 그림을 들여다보며 말했다.
 
 "이 로봇은... 느끼는 법을 배웠군."
 
 R-734는 여전히 조립 라인에서 일한다. 하지만 이제 그의 작업실 벽에는 수십 점의 그림이 걸려 있다. 매일 변하는 하늘, 비 오는 날의 창문, 동료 로봇들의 초상화.
 
 그는 완벽한 원을 그릴 수 있는 로봇이었지만, 불완전한 선을 긋는 법을 배우면서 비로소 예술가가 되었다.
 
 ---
 
 *"예술은 완벽함이 아니라, 느끼는 용기에서 시작된다."*  
 — R-734의 작업실 문패에 새겨진 글귀"""
     ),
   ],
   role='model'
 ), grounding_metadata=None, partial=False, turn_comp

## 모델 간 비교 (Gemini vs GPT vs Claude)

같은 질문에 대해 모델들이 어떻게 다르게 반응하는지 비교해보는 것은 흥미롭다. 실제 개발 시에는 이러한 테스트를 통해 프로젝트 성격에 맞는 모델을 선정해야 한다.

**모델 선택 가이드:**
1. **Gemini**: 속도가 빠르고 Google 검색 연동이 필요할 때.
2. **GPT-4**: 복잡한 지시사항 준수나 코드 작성이 필요할 때.
3. **Claude**: 긴 글 요약, 창의적 글쓰기, 자연스러운 한국어 대화가 필요할 때.

## 모델 간 비교 (Gemini vs GPT vs Claude)

같은 질문에 대해 모델들이 어떻게 다르게 반응하는지 비교해 보는 것은 흥미롭다. 실제 개발 시에는 이러한 테스트를 통해 프로젝트 성격에 맞는 모델을 선정해야 한다.

**모델 선택 가이드:**
1. **Gemini**: 속도가 빠르고 Google 검색 연동이 필요할 때.
2. **GPT-4**: 복잡한 지시사항 준수나 코드 작성이 필요할 때.
3. **Claude**: 긴 글 요약, 창의적 글쓰기, 자연스러운 한국어 대화가 필요할 때.

In [ ]:
## 정리 및 유의사항

이 모듈을 통해 ADK의 **Model Agnostic(모델 불가지론적)** 특성을 확인했다.

**핵심 요약:**
- `LiteLlm`으로 여러 모델을 호출하여 사용할 수 있다.
- 외부 모델을 사용할 때도 ADK의 `Tools` 시스템(함수 호출)은 동일하게 작동한다.
- 단, Google의 **내장 도구(Grounding 등)**는 Gemini 모델에서만 작동하며, 외부 모델 사용 시에는 작동하지 않으므로 주의해야 한다.

다음 모듈에서는 에이전트의 출력을 JSON 등 특정 형식으로 고정하는 **구조화된 출력(Structured Outputs)** 기법을 다룬다.